In [ ]:
options("scipen"=100, "digits"=4)
if(!require("readr")) install.packages("readr")
if(!require("rpart")) install.packages("rpart")
if(!require("rpart.plot")) install.packages("rpart.plot")
if(!require("Metrics")) install.packages("Metrics")
library("readr")
library("rpart")
library("rpart.plot")
library("Metrics")

Building the Decision Tree using the training set
-------------------------------------------------

So here is the data we have, this will be our training data:

-   `Purchased` is our result or outcome
-   `Salary`, `Age` are the predictors

This data is about what kinds of customers responded to an email add
offering a special deal on a product. Were they high salary or low
salary and what about their age? We would like to know if `Salary` and
`Age` of the customer help us predict whether they bought an item based
on the offer they got.

Here is our training set:

| Salary | Age | Purchased |
|:-------|:----|:----------|
| 53700  | 41  | no        |
| 65300  | 37  | no        |
| 48900  | 45  | yes       |
| 64800  | 49  | yes       |
| 69200  | 30  | yes       |
| 55900  | 57  | yes       |
| 48600  | 26  | no        |
| 72000  | 60  | yes       |
| 45300  | 34  | no        |
| 69000  | 32  | yes       |
| 73200  | 52  | yes       |

Lets read it in:

In [ ]:
trainurl<- "https://docs.google.com/spreadsheets/d/e/2PACX-1vTx21HIv4xKL2c94OLQzMiNPmv5YXw-vAy3dRBL0gnknSMKMK4Ur_4nYGGyDtvS20iU3_r-_zZfQQmj/pub?gid=0&single=true&output=csv"
traindf<-read.csv(trainurl, stringsAsFactors=TRUE)
str(traindf)

We are going to model this situation with a decision tree. We will start
by just trying to use `Salary` to predict the result:

In [ ]:
control = rpart.control(minsplit=1,maxdepth=1)
model1 <- rpart(Purchased~Salary, data=traindf, method="class", control = control)
rpart.plot(model1, type=4, extra=101, digits=-2)

What about if we use `Age` to predict the result:

In [ ]:
control = rpart.control(minsplit=1,maxdepth=1)
model2 <- rpart(Purchased~Age, data=traindf, method="class", control = control)
rpart.plot(model2, type=4, extra=101, digits=-2)

We about if we use `Salary` but let the tree grow a little:

In [ ]:
control = rpart.control(minsplit=1,maxdepth=2, cp=-1)
model3 <- rpart(Purchased~Salary, data=traindf, method="class", control = control)
rpart.plot(model3, type=4, extra=101, digits=-2)

How about if we grow the `Salary` tree even more:

In [ ]:
control = rpart.control(minsplit=1,maxdepth=3, cp=-1)
model4 <- rpart(Purchased~Salary, data=traindf, method="class", control = control)
rpart.plot(model4, type=4, extra=101, digits=-2)

How about all out with `Salary`:

In [ ]:
control = rpart.control(minsplit=1)
model5 <- rpart(Purchased~Salary, data=traindf, method="class", control = control)
rpart.plot(model5, type=4, extra=101, digits=-2)

Testing the Decision Tree model using the testing set
-----------------------------------------------------

Now lets doe some predictions using the test dataframe:

Here we read the test set:

| Salary | Age | Purchased |
|:-------|:----|:----------|
| 53900  | 45  | yes       |
| 64800  | 49  | yes       |
| 50000  | 32  | no        |
| 55900  | 57  | yes       |
| 55600  | 29  | no        |
| 60000  | 25  | no        |

In [ ]:
#read the testing set
testurl<- "https://docs.google.com/spreadsheets/d/e/2PACX-1vTx21HIv4xKL2c94OLQzMiNPmv5YXw-vAy3dRBL0gnknSMKMK4Ur_4nYGGyDtvS20iU3_r-_zZfQQmj/pub?gid=572053114&single=true&output=csv"
testdf<-read.csv(testurl, stringsAsFactors=TRUE)
str(testdf)

Lets test our last model (model5) and see how well these predictions
agree with the actual in the test set. Here is the “accuracy” for
model5:

In [ ]:
pred <- predict(model5, newdata = testdf, type = 'class')
accuracy(testdf$Purchased, pred)

Lets make a data frame that shows the prediction and the actuals for
model5:

In [ ]:
comparedf <- data.frame(actual=testdf$Purchased, predictions=pred)
print(comparedf)

Lets test model4 for its accuracy:

In [ ]:
pred <- predict(model4, newdata = testdf, type = 'class')
accuracy(testdf$Purchased, pred)

And model3:

In [ ]:
pred <- predict(model3, newdata = testdf, type = 'class')
accuracy(testdf$Purchased, pred)

And model2:

In [ ]:
pred <- predict(model2, newdata = testdf, type = 'class')
accuracy(testdf$Purchased, pred)

Finally lets build a model that uses both Salary and Age:

In [ ]:
control = rpart.control(minsplit=1)
model6 <- rpart(Purchased~Salary+Age, data=traindf, method="class", control = control)
rpart.plot(model6, type=4, extra=101, digits=-2)

Lets check the accuracy of this model that uses both variables:

In [ ]:
pred <- predict(model6, newdata = testdf, type = 'class')
accuracy(testdf$Purchased, pred)

Lets make a data frame that shows the prediction and the actuals for
this model6:

In [ ]:
comparedf <- data.frame(actual=testdf$Purchased, predictions=pred)
print(comparedf)